In [2]:
import numpy as np
import tensorflow as tf

# train test split

In [3]:
from keras.utils import image_dataset_from_directory

In [11]:
ds_train = image_dataset_from_directory('../data/french_bird_db_50_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, batch_size=64)

ds_test = image_dataset_from_directory('../data/french_bird_db_50_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, batch_size=64)

Found 5000 files belonging to 50 classes.
Using 4000 files for training.
Found 5000 files belonging to 50 classes.
Using 1000 files for validation.


In [5]:
# Obtenir le nombre de classes à partir de ds
num_classes = len(ds_train.class_names)

print('Nombre de classes :', num_classes)

Nombre de classes : 50


In [6]:

# Convertir les étiquettes cibles en vecteurs one-hot
ds_train_mapped = ds_train.map(lambda x, y: (x, tf.one_hot(y, num_classes)))
ds_test_mapped = ds_test.map(lambda x, y: (x, tf.one_hot(y, num_classes)))

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.config.list_physical_devices()

# Charger le modèle ResNet50 pré-entraîné avec les poids ImageNet
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = GlobalAveragePooling2D()(x)

x_3 = Dense(num_classes, activation='relu')(x_2)

# Ajouter une couche dense pour effectuer la classification
output = Dense(num_classes, activation='softmax')(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle de base pour éviter de les entraîner
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             TensorBoard(log_dir='./logs/relu', histogram_freq=0, write_graph=True, write_images=True)]

# Entraîner le modèle sur votre dataset avec le callback personnalisé

model.fit(ds_train_mapped, epochs=10, callbacks=callbacks, validation_data=ds_test_mapped)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.1005 - loss: 3.6099 - val_accuracy: 0.3590 - val_loss: 2.4824
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.4747 - loss: 2.1234 - val_accuracy: 0.5410 - val_loss: 1.7299
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - accuracy: 0.6211 - loss: 1.4687 - val_accuracy: 0.6370 - val_loss: 1.4189
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.7086 - loss: 1.1375 - val_accuracy: 0.6450 - val_loss: 1.2526
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.7677 - loss: 0.9091 - val_accuracy: 0.6920 - val_loss: 1.1630
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.8089 - loss: 0.7887 - val_accuracy: 0.6750 - val_loss: 1.1073
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.8320 - loss: 0.6859 - val_accuracy: 0.7000 - val_loss: 1.0424
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.8616 - loss: 0.5946 - val_accuracy: 0.7020 - v

In [9]:
model.save_weights('current_model.weights.h5')

In [13]:
import json

# Stocker les noms de classes dans une liste
class_names = ds_train.class_names

# Stocker la liste dans un fichier JSON
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

In [16]:
ds_test.labels

AttributeError: '_PrefetchDataset' object has no attribute 'labels'

In [18]:
def extract_label(image, label):
    return label

# Mapper chaque élément du jeu de test à son étiquette correspondante
ds_test_labels = ds_test.map(extract_label)

# Convertir les étiquettes en tableau NumPy
labels_list = list(ds_test_labels.unbatch())
y_true_classes = np.array(labels_list)
# y_true_classes = tf.concat([ds_test_labels.batch(ds_test.cardinality().numpy()).unbatch().numpy()], axis=0)

2024-07-02 15:41:42.031579: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(ds_test)


16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step


In [23]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_pred_classes

array([45, 24, 15, 45, 30, 20, 35, 16, 37, 33, 45,  9, 35,  9, 45, 19, 22,
       35, 18, 11, 41, 48, 37, 46, 49,  8, 17,  8,  0, 22,  2, 26, 12, 37,
       10, 30,  3, 36, 24, 44, 23,  6, 27, 25, 19, 34, 19, 39,  2, 37, 49,
       37, 20, 39, 13,  8, 40, 27, 21, 23, 10,  4,  6, 46, 45,  2, 49, 35,
        3,  0, 43,  7, 27,  1, 23, 11, 36, 39, 45,  2,  6, 38, 14, 22, 48,
       48, 32, 14, 35, 45, 25, 27, 11, 17, 23,  7, 27, 19,  1, 18, 42, 37,
       24, 19, 25, 18, 29, 46, 45,  2,  6, 28, 17,  0, 32, 28, 39, 13, 48,
        1, 11, 23, 20, 36, 14, 25,  0, 32, 10, 11, 37, 37, 24, 43, 19, 48,
        2, 15, 37, 35, 13, 35,  9, 11, 22, 47,  3, 48, 44, 33, 18,  2, 19,
       42, 29, 13,  9,  1, 39, 35, 49, 31, 41, 37, 43,  3, 22,  7, 34, 36,
       13,  1,  7, 33, 26, 43, 33, 39, 10, 21,  1,  0, 22, 39, 29,  0, 48,
        9, 29, 31, 15, 13, 10, 29, 27, 40,  1, 30,  1, 37, 31, 30, 48, 33,
       42, 27, 43, 45, 43, 43,  1,  1, 22, 16, 26,  1,  3, 22,  5,  8,  0,
       32, 42, 10, 27,  6

In [26]:
# Définir une fonction pour extraire l'étiquette d'un élément du jeu de test
def extract_label(image, label):
    return label

# Mapper chaque élément du jeu de test à son étiquette correspondante
ds_test_labels = ds_test.map(extract_label)
labels_list = list(ds_test_labels.unbatch())
y_true = np.array(labels_list)
y_true

2024-07-02 15:47:19.045237: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


array([23, 20, 47, 11, 22, 26, 29, 20, 44,  5, 29, 42,  4,  9, 27,  8, 48,
        4, 21, 44, 12,  1, 33, 46, 22, 28, 16, 13,  3, 45,  3,  9, 45,  9,
        0, 27, 43, 18, 35, 12, 29, 44,  0, 12,  1, 40,  8, 27, 41, 34, 41,
        9, 34, 36, 35, 26, 48, 34, 19, 25, 27, 34, 35, 11,  4, 23, 18, 16,
       20, 32, 38, 10, 46,  5, 21, 27, 13, 31, 26, 37, 30,  5,  1, 27,  7,
        3, 18, 25, 21, 32,  6, 15, 33, 22, 36,  6, 30, 26,  9, 33, 41, 41,
       12, 27, 28, 29, 28, 40, 15, 13, 41,  9, 32, 34, 35,  9, 13, 15, 28,
       47, 16,  3, 16, 45, 27, 49, 43,  9,  9, 20, 22, 40, 39, 19, 48,  6,
        0, 32, 40, 33, 14, 11, 49, 45, 29, 43, 39, 21, 32, 11, 21, 28,  9,
       10,  6, 13, 44, 11, 17, 34, 11, 13, 38,  4, 48, 19, 33,  1, 41, 42,
       18, 47, 16, 37, 44,  2,  7, 13, 34, 49, 40, 12, 30, 12, 10, 36, 46,
        2, 24, 38, 23, 41, 35, 33, 29, 41,  3,  4, 43, 11, 30, 49, 30, 35,
       45, 32, 33,  4, 35, 44, 15, 16, 31, 31, 21,  7,  8, 42, 26, 37,  7,
       18, 40,  8,  4,  2

In [ ]:

# y_true_classes = np.argmax(ds_test.labels, axis=1)
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)